# Hashing and Spam
#### Article Classification(E.g: Spam Classification) using bag-of-words representation (BOW)

Suppose we have two sentence one is span and other is not spam   
- i earn 20 lakh rupees per month just chitchating on the net!(spam)  
- are you free for a meeting anytime tomorrow?(not spam) 


In [1]:
import pandas as pd
corpus = [
     'i earn twenty lakh rupees per month just chitchating on the net!', 
     'are you free for a meeting anytime tomorrow?',
]
df = pd.DataFrame({'Text':corpus})
from sklearn.feature_extraction.text import CountVectorizer
count_v = CountVectorizer()
X = count_v.fit_transform(df.Text).toarray()
print(X)
print(count_v.vocabulary_)


[[0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0]
 [1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1]]
{'earn': 3, 'twenty': 16, 'lakh': 7, 'rupees': 13, 'per': 12, 'month': 9, 'just': 6, 'chitchating': 2, 'on': 11, 'the': 14, 'net': 10, 'are': 1, 'you': 17, 'free': 5, 'for': 4, 'meeting': 8, 'anytime': 0, 'tomorrow': 15}


In [2]:
#Now we got a new sentence, generate the vector using pre built vocabulary
new_txt = ['io etrn are you free ruppee for a monnth meeting chitcchting anytime tomorrow neet']
df_new = pd.DataFrame({'new_txt':new_txt})
y = count_v.transform(df_new.new_txt).toarray()
print(y)


[[1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1]]


Vocabulary:  
{'anytime': 0, 'are': 1, 'chitchating': 2, 'earn': 3, 'for': 4, 'free': 5, 'just': 6, 'lakh': 7, 'meeting': 8,   'month': 9, 'net': 10, 'on': 11, 'per': 12, 'rupees': 13, 'the': 14, 'tomorrow': 15, 'twenty': 16, 'you': 17 }  
  
- i earn 20 lakh rupees per month just chitchating on the net!   
vector1:[0 0 1 1 0 0 1 1 0 1 1 1 1 1 1 0 1 0]  

- are you free for a meeting anytime tomorrow?  
vector2:[1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1]  


New Sentence:  
- io etrn are you free ruppee for a monnth meeting chitcchting anytime tomorrow neet  
With Existing Count Vector:  
vector3:[1 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 1]  


- So, above vector3 is same as vector2, because new words can't be taken into account as this is CountVectorizer. So machine learning will classify vector3 as not spam same as vector2 which is not correct. 
- Even if we increase the vector length, whole machine learning model need to be trained again.  
- Solution is Hashing


## Hashing
- Apply Hash Function
- “Rishi Bansal”  --> 23
- “Rashi Bansal” --> 72
- Output number depend on Hash function


**Features:**
- Same value for same string
- Collison: Possibility of same value for different string 


Lets take a huge count vector – 2*2^20  
Choose Hashing function which generates number between 0 to 2*2^20  
'i earn 20 lakh rupees per month just chitchating on the net!',   
[0 0 1 1…. 0 0 1 1 0….. 1 1 1 1 ……1 1 0 1 0]  – 2*2^20  

 'are you free for a meeting anytime tomorrow?',  
[1 1 …..0 0 1 1 0 0 ……1 0……. 0 0 0 0 …..0 1 0 1]  – 2*2^20  

 'io etrn are you free ruppee for a monnth meeting chitcchting anytime tomorrow neet'  
[00 …..1 0 1 1 0 0 ……1 0……. 1 0 1 0 …..0 0 0 1] – 2*2^20   



## When to Use HashVectorizer?

#### HashingVectorizer : 
1> If dataset is large and there is no use for the resulting dictionary of tokens
2> You have maxed out your computing resources and it’s time to optimize

#### CountVectorizer: 
1>  If need is to access the actual tokens.
2> If you are worried about hash collisions (when matrix size is small) 


## Spam Filter

In [1]:
import pandas as pd
import numpy as np

dataset = pd.read_csv("Data/emails.csv")

X = dataset["text"]
y = dataset["spam"]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

from sklearn.feature_extraction.text import CountVectorizer
spam_fil = CountVectorizer(stop_words="english")
#spam_fil.fit(X_train) 
#print(spam_fil.get_feature_names())

X_train = spam_fil.fit_transform(X_train).toarray()
X_test = spam_fil.transform(X_test).toarray()
X_train


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [3, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [2]:
#using K-Nearest Neighbors
from sklearn.neighbors import KNeighborsClassifier
kneigh = KNeighborsClassifier(n_neighbors = 5)
kneigh.fit(X_train, y_train)

from sklearn.metrics import accuracy_score, confusion_matrix
pred_test = kneigh.predict(X_test)
print(accuracy_score(y_test, pred_test))

0.9022687609075044


In [21]:
#Using Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

pred_test_nb = nb_model.predict(X_test)
print(accuracy_score(y_test, pred_test_nb))

0.9505526468877254


In [25]:
#Using Naive Bayes Multinomial
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
mnb_model = MultinomialNB(alpha=1.8)
mnb_model.fit(X_train.toarray(), y_train)
pred_test_mnb = mnb_model.predict(X_test.toarray())
accuracy_score(y_test, pred_test_mnb)

0.9936009307737056